In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns #; sns.set_theme() # For heat map
from scipy.optimize import minimize # For automatic minimisation of the negative
# likelihood of the position
from ipynb.fs.full.class_virt_sensor import virt_sensor_net2
from ipynb.fs.full.class_ship import ship_fleet
from ipynb.fs.full.class_black_sea import black_sea_obj
import numpy.random as rnd
import numpy.linalg as linalg
import matplotlib.path as mpltPath # to check if the point is on the border
from scipy.optimize import NonlinearConstraint

In [ ]:
# NOTE: Need to define the black sea object before reading the methods in this notebook

black_sea_coords = np.load('blackSea_polygon_coords.npy')
global black_sea
black_sea = black_sea_obj(black_sea_coords)

In [10]:
# CURRENT VERSION: The function to be optimised w.r.t. the sensor placement
# (or rather minimised) to be the mean of the average value of all ship lane 
# prediction errors where each ship lane is repeated num_rep times to account 
# for random errors in the automated gradient descent w.r.t. the likelihood 
# maximisation procedure.

# NEXT VERSION: Add a function that approximates the gradient of the above 
# function by moving each sensor individual in each direction slightly, repeating
# the entire mean average measuring process, and then using this insight to
# approximate the gradient of the entire network

In [11]:
# LIKELIHOOD (virtual sensors)

def nlhd(xy, network, ship_idx): # Negative likelihood at point xy for ship "ship_idx"
    lh_val = - network.get_net_likelihood(ship_idx, xy) # Negative network likelihood
    return lh_val

In [12]:
# MAIN GRADIENT DESCENT METHOD (virtual sensors): To be optimised (minimised) MSE loss

def MSE_loss(sensor_coords): # MSE loss minimising sensor placement
    count = 0
    time = 0
    network = virt_sensor_net2(sensor_coords.reshape(m,2))
    for i in range(fleet.size):
        
        while(count < num_rep):
            
            prev_est = rnd.uniform(0, .2, size = (2,))
            while(time < time_stop):
            
                fleet.update_fleet_positions(time) # Update the position of all ships
                if(time == 0):
                    lane_actual = fleet.coords[i]
                else:
                    lane_actual = np.vstack((lane_actual, fleet.coords[i])) # Save position for lane MSE
                network.def_gaussian_spheres(fleet, err_mean, err_stdev) # Update
                # network Gaussian spheres for new fleet position

                xy_0 = prev_est
                coord_pred_at_time = minimize(nlhd, xy_0, args = (network, i),
                                              method='nelder-mead',
                                              options={'xatol': 1e-4}).x
                if(time == 0):
                    lane_pred = coord_pred_at_time
                else:
                    lane_pred = np.vstack((lane_pred,coord_pred_at_time))
                prev_est = coord_pred_at_time
                time += 1 
                
            if(count == 0):
                lane_i_error = np.mean(linalg.norm(lane_actual - lane_pred)**2)
            else:
                lane_i_error = np.vstack((lane_i_error, np.mean(linalg.norm(lane_actual - lane_pred)**2)))
            count += 1
            
        if(i==0):
            MSE_for_lane_i = np.mean(lane_i_error)
        else:
            MSE_for_lane_i = np.vstack((MSE_for_lane_i, np.mean(lane_i_error)))

    avg_MSE_over_all_lanes = np.mean(MSE_for_lane_i)
    
    return avg_MSE_over_all_lanes

In [13]:
# ARGUMENT FOR GRADIENT DESCENT (virtual sensors): Pseudo gradient at the current position

def pseudo_MSE_loss_gradient(sensor_coords): # Shifting individual sensors, and
# repetition of measuring to define a gradient approximation in the point
    
    eps = .05 # Position change in EACH of the (x,y) directions
    sensor_coords = sensor_coords.reshape(m, 2)
    
    pseudo_loss_gradient = []
    for i in range(m):
        x_i_eps_shift = np.zeros((m,2))
        x_i_eps_shift[i, 0] += eps
        sensor_coords_x_i_plus_eps = sensor_coords + x_i_eps_shift
        sensor_coords_x_i_minus_eps = sensor_coords - x_i_eps_shift
        loss_gradient_x_i = (MSE_loss(sensor_coords_x_i_plus_eps) - MSE_loss(sensor_coords_x_i_minus_eps))/eps
        
        y_i_eps_shift = np.zeros((m,2))
        y_i_eps_shift[i, 1] += eps
        sensor_coords_y_i_plus_eps = sensor_coords + y_i_eps_shift
        sensor_coords_y_i_minus_eps = sensor_coords - y_i_eps_shift
        loss_gradient_y_i = (MSE_loss(sensor_coords_y_i_plus_eps) - MSE_loss(sensor_coords_y_i_minus_eps))/eps
        
        pseudo_loss_gradient.append(loss_gradient_x_i) # Output needs to be a gradient vector in the form (dx_1, dy_1, dx_2, dy_2,...)
        pseudo_loss_gradient.append(loss_gradient_y_i)
    
    return np.asarray(pseudo_loss_gradient)

In [14]:
# ARGUMENT FOR GRADIENT DESCENT (virtual sensors): Will be used to define the contraint given to the SLSQP method
def within_borders(sensor_coords):
    # Using a path over the black sea coordinates defined in "black_sea.border_points"
    # to check if ALL points (written as matrix) are still within the borders
    
    # PROBLEM: Global definition of black_sea before reading the method does NOT suffice to define the black sea object
    
    m = int(len(sensor_coords.reshape(-1,1))/2) # Need it for if-condition below
    sensor_coords = sensor_coords.reshape(m,2)    
    path = mpltPath.Path(black_sea.border_points)
    
    if (m == 1):# ATTENTION: For multiple points we need "path.contains_points"
    # and then it only works for multiple points but not for a single one
        tf_value = path.contains_point(sensor_coords.reshape(-1))
        if tf_value:
            return 1
        else:
            return -1
    else:
        tf_value = path.contains_points(sensor_coords)    
        if min(tf_value): # tf_value is a vector of True/False; we only allow all True
            return 1
        else:
            return -1

In [2]:
# Method that chooses optimal number of sensors based on AIC criterion


# ToDo: Need method that places sensors randomly in the allowed area; IDEA:
# using acceptance-rejection, the method draws uniform random variables until
# it has the number of needed sensors

def random_sensor_placement(number_sensors):
    n = number_sensors
    sensor_coords = []
    while (len(sensor_coords) < n):
        trial_sensor = rnd.uniform(0, 1, size = 2)
        if (within_borders(trial_sensor) == 1):
            sensor_coords.append(trial_sensor)
    return np.array(sensor_coords).reshape(-1)


def calc_model_loss(mse_loss, complexity, lambd):
    '''
    # AIC criterion approach
    k = 2*complexity # Number of parameter estimates (as the total number of xy coordinates)
    rating = 2*k - np.log(mse_loss) #PROBLEM: MSE loss is <<1 since we do not work
    # with likelihood functions (which is assumed by Akaike); FIX: Employ a
    # Lasso/Ridge like approach by penalising complexity by additive loss
    '''
    # Lasso/Ridge approach
    model_loss = mse_loss + lambd*complexity
    return model_loss

def get_optimal_sensor_number(max_num):
    log_book = [[],[],[]] # this log book should hold the information of the procedure
    # where the first component holds the initial sensor placements, the second
    # the "optimal" sensor coordinates after optimisation, and the last should
    # hold the rating of the model under optimality; this is continuously written
    # over changing m = 1,...,max_num
    global m
    m = 1
    best_number = m # We will change the optimal number of sensors iteratively
    # from one iteration to the next ONLY if the new number has a lower model loss
    best_model_loss = 1e5 # Big initial loss to start overwriting
    # FOR ANALYSIS: Also save the worst number of sensors
    worst_number = 1
    worst_model_loss = 0 
    while (m < max_num):
        coords_init = random_sensor_placement(m)
        bnds = (((0,1),) * 2*m) # Need to define 2m bounds for each element of the variable vector
        # NOTE: 'ftol' is the termination tolerance w.r.t. the gradient at the points (since we want to minimise)
        res = minimize(MSE_loss, coords_init, method = 'SLSQP', jac = pseudo_MSE_loss_gradient,
               bounds = bnds, constraints = nlc, options={'ftol': 1e-10, 'maxiter': 200, 'disp': True})
        coords_opt = res.x
        mse_loss = res.fun
        model_loss = calc_model_loss(mse_loss, m, .001) # TODO: Decide on optimal lambda value
        # through cross-validation ?; for now, it is only fixed
        if (model_loss < best_model_loss):
            best_model_loss = model_loss
            best_number = m
            
        # FOR ANALYSIS:
        if (model_loss > worst_model_loss):
            worst_number = m
            worst_model_loss = model_loss
        
        m += 1
        log_book[0].append(coords_init)
        log_book[1].append(coords_opt)
        log_book[2].append(model_loss)
    
    return best_number, worst_number, log_book

In [16]:
# FOR OUTPUT
# For prediction output: Re-do the repetitive likelihood prediction for the 
# optimal network coordinates and plot the mean predictions
    
def MSE_loss_plot(sensor_coords): # MSE loss minimising sensor placement
    network = virt_sensor_net2(sensor_coords.reshape(m,2))
    for i in range(fleet.size):
        count = 0
        while(count < num_rep):
            time = 0
            # PROBLEM with random initial value: Too much of a wild card
            '''
            prev_est = rnd.uniform(10,40, size = (2,))
            '''
            # CURRENT VERSION: Set initial start point for negative likelihood
            # minimisation by hand
            
            prev_est = np.array([.2, .2])
            
            while(time < time_stop):
            
                fleet.update_fleet_positions(time) # Update the position of all ships
                if(time == 0):
                    lane_actual = fleet.coords[i]
                else:
                    lane_actual = np.vstack((lane_actual, fleet.coords[i])) # Save position for lane MSE
                network.def_gaussian_spheres(fleet, err_mean, err_stdev) # Update
                # network Gaussian spheres for new fleet position

                xy_0 = prev_est
                # PROBLEM: 'Nelder-Mead' approach does not allow for bounds w.r.t.
                # the position
                
                coord_pred_at_time = minimize(nlhd, xy_0, args = (network, i),
                                              method='nelder-mead',
                                              options={'xatol': 1e-8}).x
                
                # TRIAL: Use 'SLSQP' algorithm where we bound the values onto the
                # space of interest
                '''
                bnds = ((0, 50),(0, 50))
                coord_pred_at_time = minimize(nlhd, xy_0, args = (network, i),
                                              method='SLSQP',
                                              bounds = bnds).x
                '''
                if(time == 0):
                    lane_pred = coord_pred_at_time
                else:
                    lane_pred = np.vstack((lane_pred,coord_pred_at_time))
                prev_est = coord_pred_at_time
                time += 1 
                
            if(count == 0):
                lane_i_error = np.mean(linalg.norm(lane_actual - lane_pred)**2)
                # OUTPUT CHANGE IS HERE:
                # For illustration: Save lane prediction now and add others later
                # to get easy mean estimate afterwards
                test_mean_lane_pred = lane_pred
            else:
                lane_i_error = np.vstack((lane_i_error, np.mean(linalg.norm(lane_actual - lane_pred)**2)))
                # OUTPUT CHANGE IS HERE:
                test_mean_lane_pred += lane_pred
            count += 1
            
        if(i==0):
            MSE_for_lane_i = np.mean(lane_i_error)
            # OUTPUT CHANGE IS HERE:
            # Save the mean lane predictions as list of arrays
            test_lane_pred = [num_rep**(-1) * test_mean_lane_pred]
            test_actual_lanes = [lane_actual]
        else:
            MSE_for_lane_i = np.vstack((MSE_for_lane_i, np.mean(lane_i_error)))
            # OUTPUT CHANGE IS HERE:
            # For test: Save the mean lane predictions as list of arrays
            test_lane_pred.append(num_rep**(-1) * test_mean_lane_pred)
            test_actual_lanes.append(lane_actual)

    avg_MSE_over_all_lanes = np.mean(MSE_for_lane_i)
    
    print("The average MSE over all lanes and repetitions is: " + str(avg_MSE_over_all_lanes))
    # ONLY for test: Print the ships, sensors and their predictions
    fig, ax = plt.subplots()
    ax.plot(network.coords[:,0], network.coords[:,1], 'b^')
    for i in range(fleet.size):
        fig = plt.gcf()
        ax = fig.gca()
        ax.plot(test_actual_lanes[i][:,0], test_actual_lanes[i][:,1], 'ro')
        ax.plot(test_lane_pred[i][:,0], test_lane_pred[i][:,1], 'go')